In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
from urllib.parse import urlparse, urlencode, urlunparse, parse_qs

In [2]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from tqdm import tqdm

In [3]:
used = ["https://www.olx.ua/uk/nedvizhimost/kvartiry/prodazha-kvartir/kiev/q-%D0%BA%D0%B2%D0%B0%D1%80%D1%82%D0%B8%D1%80%D0%B0/?currency=USD&page=1&search%5Bdistrict_id%5D=19&search%5Bfilter_enum_apartments_object_type%5D%5B0%5D=secondary_market",
        "https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=0&d_id=15190&ch=242_239,247_252"]
new = ["https://www.olx.ua/uk/nedvizhimost/kvartiry/prodazha-kvartir/kiev/q-%D0%BA%D0%B2%D0%B0%D1%80%D1%82%D0%B8%D1%80%D0%B0/?currency=USD&page=1&search%5Bdistrict_id%5D=19&search%5Bfilter_enum_apartments_object_type%5D%5B0%5D=primary_market",
      "https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&newbuildings=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=0&d_id=15190&ch=242_239,247_252"]

In [28]:
data = []

In [5]:
# Тут ми дізнаємось кількість сторінок для посилання: new[1]
req = requests.get(new[0])
soup = BeautifulSoup(req.text, 'lxml')
num_of_page = soup.findAll("a", class_ = "css-1mi714g")[3].text
num_of_page

'24'

In [18]:
# Це парсер для olx, вся інформація буде записуватись в data_olx змінну 
link = new[0]
parsed_url = urlparse(link)
query_dict = parse_qs(parsed_url.query)
for i in range(int(num_of_page)):
    print(i+1) # інформативне виведення поточної сторінки
#    тут використовується urllib для зміни номеру сторінки в url
    query_dict['page'] = [str(i+1)]
    updated_query = urlencode(query_dict, doseq=True)
    updated_url = urlunparse(parsed_url._replace(query=updated_query))
    
    req = requests.get(updated_url)
    soup = BeautifulSoup(req.text, 'lxml')
    div_res = soup.find('div', class_ ="css-oukcj3").findAll('div', class_ = "css-1sw7q4x")
    
#     Парсинг однієї сторінки
    for one_piece in div_res:  
        price = one_piece.find('p', class_ = "css-10b0gli er34gjf0").text
        square = one_piece.find('span', class_ ="css-643j0o").text
        name = one_piece.find('h6', class_ = "css-16v5mdi er34gjf0").text
    #     pictr_link = one_piece.find('div', class_ ="css-gl6djm").find('img').get('src')
    #     location_data = one_piece.find('p', class_ = "css-veheph er34gjf0").text
        link = 'https://www.olx.ua' + one_piece.find('a', class_ = "css-rc5s2u").get('href')

        req = requests.get(link)
        soup = BeautifulSoup(req.text, 'lxml')
        categories = soup.findAll(class_="css-1r0si1e")
        bool_swtch = True
        for category in categories:
            if "Поверх" in category.text and bool_swtch == True:
                part = category.text.split(':')
                poverh = part[1].strip( )
                bool_swtch = False
            if "Кількість кімнат" in category.text:
                part = category.text.split(':')
                num_of_room = part[1].split(' ')[1]

        data_olx.append([name, price, square, num_of_room, poverh])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [11]:
data_olx

[['Продається 2-кімнатна квартира в історичному центрі Києва. Сталінка.',
  '106 900 $Договорная',
  '44 м²',
  '3',
  '2'],
 ['Євродвушка в ЖК "Нивки-парк" - пр.Перемоги 65-Д. Підходить під  ЄОселя',
  '116 000 $',
  '59 м²',
  '3',
  '2'],
 ['Продаж 2 кімн. квартири за адресою вул. Бульварно-Кудрявська 31а',
  '214 000 $',
  '78 м²',
  '3',
  '2'],
 ["Квартира, 1к. Багговутівська, 1а, Лук'янівка, Шевченківський район",
  '40 000 $Договорная',
  '37 м²',
  '3',
  '2'],
 ['Продажа 2 ком.кв.40кв м.ст.м."Дорогожичи" ул.Орловская4-6',
  '49 999 $',
  '40 м²',
  '3',
  '2'],
 ['1к квартира Київ Поділ центр', '43 000 $', '31 м²', '3', '2'],
 ["Квартира, 1к. Білоруська, 21, Лук'янівка, Шевченківський район",
  '60 000 $Договорная',
  '41 м²',
  '3',
  '2'],
 ['2к квартира, 47м2, Пр.Перемоги(Берестейський) 3, під ремонт',
  '51 000 $',
  '47 м²',
  '3',
  '2'],
 ['Продається 2-к квартира  у центрі міста.Метро Вокзальна',
  '92 000 $',
  '50 м²',
  '3',
  '2'],
 ['Продаж, 1к квартира. ЖК «Podi

In [24]:
# Запуск селеніум
path = "C:\Program Files (x86)\chromedriver.exe"
driver = Chrome() 
driver.get(used[1])

In [25]:
# Закриття вікна на сайті
X_form = driver.find_element(By.CSS_SELECTOR, 'body > div.modal-mask.modal-onboarding-verified.search > div.modal-wrapper > div > div.modal-header.p-rel > h3 > a')
X_form.click()

In [29]:
# Код для парсингу domria
page_content = driver.page_source
soup = BeautifulSoup(page_content, 'lxml')

# Тут ми дізнаємось кількість сторінок для посилання: used[1] та перетворюємо в int
num_of_page = soup.findAll('a', class_="page-item button-border")[-1].text
int_num_of_page = int(''.join(filter(str.isdigit, num_of_page)))
for i in range(int_num_of_page):
#   тут використовується urllib для зміни номеру сторінки в url
    parsed_url = urlparse(used[1])
    query_params = parse_qs(parsed_url.query)
    query_params['page'] = [i]
    new_url = urlunparse(parsed_url._replace(query=urlencode(query_params, doseq=True)))
    print(new_url)
    try:
        driver.get(new_url)
        sleep(2)# Використовується для того, щоб сторінка встигла прогрузитись та задля запобігання підозрілої активності
        page_content = driver.page_source
        soup = BeautifulSoup(page_content, 'lxml')

        bs_page = soup.find_all('section', class_="realty-item")
        for piece in bs_page:
            price = piece.find('b', class_="size22").text
            items = piece.findAll('span', class_ ="point-before")
            num_of_room = items[1].text.split(' ')[0]
            sqr = items[2].text
            poverh = items[3].text.split(' ')[0]
            data.append([None, price, sqr, num_of_room, poverh])
    except Exception as e:
        print(i, e)

https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=0&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=1&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=2&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&cli

https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=31&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=32&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=33&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&

https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=62&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=63&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=64&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&

https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=94&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=95&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=96&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&

https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=125&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=126&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=127&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=

https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=156&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=157&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=158&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=

https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=187&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=188&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=189&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=

https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=218&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=219&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=220&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=

https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=249&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=250&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=10&client=searchV2&page=251&d_id=15190&ch=242_239%2C247_252
https://dom.ria.com/uk/search/?noSoldOut=1&category=1&realty_type=2&operation=1&state_id=10&price_cur=1&wo_dupl=1&sort=inspected_sort&secondary=1&firstIteraction=false&limit=20&market=2&type=list&city_ids=

In [34]:
new_data = data + data_olx

In [52]:
len(arr)

6643

In [20]:
data_csv = pd.read_csv('./parsing_hat_anya_used.csv', sep = ',', encoding='utf-8')
data_csv.tail(1280)

,Unnamed: 0,name,price,square,num_of_room,poverh
5363,5363,NaN,115 000 $,50 м²,1,8
5364,5364,NaN,206 800 $,88 м²,2,5
5365,5365,NaN,142 500 $,57 м²,2,2
5366,5366,NaN,1 200 000 $,180 м²,6,1
5367,5367,Продажа 3к квартиры в ЖК Пушкинский парк,330 000 $Договорная,160 м²,3,14
...,...,...,...,...,...,...
6638,6638,3 кімн в центрі Тургенівська 52/58,300 000 $,160 м²,1,4
6639,6639,"ЖК Файна Таун, 2 кімнатна, 48м2, видова, новий...",170 000 $,48 м²,1,4
6640,6640,"Без% Дизайнерская 2к 64м2 Гончара 75, м. Униве...",135 000 $,64 м²,1,4
6641,6641,"Видова 2-х кім кв. у Центрі, Львівська Площа, ...",185 000 $,80 м²,1,4


In [29]:
# Запис інформації у csv файл з заданими колонками
header = ["name", "price", "square", "num_of_room", "poverh"]
df_2 = pd.DataFrame(new_data, columns=header)